In [ ]:
!pip install djitellopy
#importing libraries
from djitellopy import Tello
import cv2
import pygame
from pygame.locals import *
import numpy as np
import time
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import fastai
from fastai.vision import * #importing libraries
from fastai.metrics import error_rate


     |████████████████████████████████| 11.4MB 4.2MB/s 
  Created wheel for djitellopy: filename=djitellopy-1.5-cp36-none-any.whl size=6387 sha256=e13954b286523bddece2278ef8af751ddc9e9e21337c2d8f6794fb4bdc1428bf
  Stored in directory: /root/.cache/pip/wheels/7b/ab/1e/5fc14755a215d23fbbe70dbb895836c8fadb2cb2d9f1412c0a
Successfully built djitellopy
pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Preparing ML model
path = '/content/drive/My Drive/1. Fight Unusual Behavior in School/Unusual Behaviour Dataset/Model2_Dataset'
data  = ImageDataBunch.from_folder(path, ds_tfms=get_transforms(), size=244, bs=16, valid_pct=0.3).normalize(imagenet_stats)
learn = cnn_learner(data, models.resnet34, metrics=accuracy)
learn.load('violence_2019')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 120MB/s]


Learner(data=ImageDataBunch;

Train: LabelList (729 items)
x: ImageList
Image (3, 244, 244),Image (3, 244, 244),Image (3, 244, 244),Image (3, 244, 244),Image (3, 244, 244)
y: CategoryList
Non-Violence,Non-Violence,Non-Violence,Non-Violence,Non-Violence
Path: /content/drive/My Drive/1. Fight Unusual Behavior in School/Unusual Behaviour Dataset/Model2_Dataset;

Valid: LabelList (312 items)
x: ImageList
Image (3, 244, 244),Image (3, 244, 244),Image (3, 244, 244),Image (3, 244, 244),Image (3, 244, 244)
y: CategoryList
Non-Violence,Non-Violence,Violence,Violence,Non-Violence
Path: /content/drive/My Drive/1. Fight Unusual Behavior in School/Unusual Behaviour Dataset/Model2_Dataset;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilatio

In [ ]:
#Setting up drone for control
# Speed of the drone
S = 60
# Frames per second of the pygame window display
FPS = 25


class FrontEnd(object):
    """ Maintains the Tello display and moves it through the keyboard keys.
        Press escape key to quit.
        The controls are:
            - T: Takeoff
            - L: Land
            - Arrow keys: Forward, backward, left and right.
            - A and D: Counter clockwise and clockwise rotations
            - W and S: Up and down.
    """

    def __init__(self):
        # Init pygame
        pygame.init()

        # Creat pygame window
        pygame.display.set_caption("Tello video stream")
        self.screen = pygame.display.set_mode([960, 720])

        # Init Tello object that interacts with the Tello drone
        self.tello = Tello()
        self.tello.streamon()
        #For storing frames to be accessed later for classification
        self.tello.streamon()
        count = 0
        vidcap = cv2.VideoCapture(self.tello.get_udp_video_address())
        success,image = vidcap.read()
        success = True
        while success:
            vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))    # added this line
            success,image = vidcap.read()
            print ('Read a new frame: ', success)
            cv2.imwrite("./img/frame%d.jpg" % count, image)     # save frame as JPEG file
            count = count + 25

        # Drone velocities between -100~100
        self.for_back_velocity = 0
        self.left_right_velocity = 0
        self.up_down_velocity = 0
        self.yaw_velocity = 0
        self.speed = 10

        self.send_rc_control = False

        # create update timer
        pygame.time.set_timer(USEREVENT + 1, 50)

    def run(self):

        if not self.tello.connect():
            print("Tello not connected")
            return

        if not self.tello.set_speed(self.speed):
            print("Not set speed to lowest possible")
            return

        # In case streaming is on. This happens when we quit this program without the escape key.
        if not self.tello.streamoff():
            print("Could not stop video stream")
            return

        if not self.tello.streamon():
            print("Could not start video stream")
            return

        frame_read = self.tello.get_frame_read()


        should_stop = False
        while not should_stop:

            for event in pygame.event.get():
                if event.type == USEREVENT + 1:
                    self.update()
                elif event.type == QUIT:
                    should_stop = True
                elif event.type == KEYDOWN:
                    if event.key == K_ESCAPE:
                        should_stop = True
                    else:
                        self.keydown(event.key)
                elif event.type == KEYUP:
                    self.keyup(event.key)

            if frame_read.stopped:
                frame_read.stop()
                break

            self.screen.fill([0, 0, 0])
            self.tello.streamon()
            count = 0
            vidcap = cv2.VideoCapture(self.tello.get_udp_video_address())
            success,image = vidcap.read()
            success = True
            while success:
                vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))    # added this line
                success,image = vidcap.read()
                print ('Read a new frame: ', success)
                cv2.imwrite("./img/frame%d.jpg" % count, image)     # save frame as JPEG file
                count = count + 25

            frame = cv2.cvtColor(frame_read.frame, cv2.COLOR_BGR2RGB)
            frame = np.rot90(frame)
            frame = np.flipud(frame)
            frame = pygame.surfarray.make_surface(frame)
            self.screen.blit(frame, (0, 0))
            pygame.display.update()

            time.sleep(1 / FPS)

        # Call it always before finishing. I deallocate resources.
        self.tello.end()

    def keydown(self, key):
        """ Update velocities based on key pressed
        Arguments:
            key: pygame key
        """
        if key == pygame.K_UP:  # set forward velocity
            self.for_back_velocity = S
        elif key == pygame.K_DOWN:  # set backward velocity
            self.for_back_velocity = -S
        elif key == pygame.K_LEFT:  # set left velocity
            self.left_right_velocity = -S
        elif key == pygame.K_RIGHT:  # set right velocity
            self.left_right_velocity = S
        elif key == pygame.K_w:  # set up velocity
            self.up_down_velocity = S
        elif key == pygame.K_s:  # set down velocity
            self.up_down_velocity = -S
        elif key == pygame.K_a:  # set yaw clockwise velocity
            self.yaw_velocity = -S
        elif key == pygame.K_d:  # set yaw counter clockwise velocity
            self.yaw_velocity = S

    def keyup(self, key):
        """ Update velocities based on key released
        Arguments:
            key: pygame key
        """
        if key == pygame.K_UP or key == pygame.K_DOWN:  # set zero forward/backward velocity
            self.for_back_velocity = 0
        elif key == pygame.K_LEFT or key == pygame.K_RIGHT:  # set zero left/right velocity
            self.left_right_velocity = 0
        elif key == pygame.K_w or key == pygame.K_s:  # set zero up/down velocity
            self.up_down_velocity = 0
        elif key == pygame.K_a or key == pygame.K_d:  # set zero yaw velocity
            self.yaw_velocity = 0
        elif key == pygame.K_t:  # takeoff
            self.tello.takeoff()
            self.send_rc_control = True
        elif key == pygame.K_l:  # land
            self.tello.land()
            self.send_rc_control = False

    def update(self):
        """ Update routine. Send velocities to Tello."""
        if self.send_rc_control:
            self.tello.send_rc_control(self.left_right_velocity, self.for_back_velocity, self.up_down_velocity,
                                       self.yaw_velocity)
    def cap(self):
        count = 0
        self.vidcap = cv2.VideoCapture('rtsp://192.168.10.1/8889')
        success,image = self.vidcap.read()
        success = True
        while success:
            self.vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))    # added this line
            success,image = self.vidcap.read()
            print ('Read a new frame: ', success)
            cv2.imwrite("./img/frame%d.jpg" % count, image)     # save frame as JPEG file
            count = count + 5

def main():
    frontend = FrontEnd()

    # run frontend
    frontend.run()


if __name__ == '__main__':
    main()

In [ ]:
img = open_image('/content/drive/My Drive/1. Fight Unusual Behavior in School/Unusual Behaviour Dataset/fr.png')
pred_class,pred_idx,outputs = learn.predict(img)
img.show()
def on_button_click(e):
    print ("Detected", pred_class)
learn.predict(img)
button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success',
    tooltip='Submit')
# Sending sms alert if violence is detected
if outputs[1]>0.5: #as seen in above code we are using probability of class as condition for sending text sms
    client.messages.create(to="add_number_here",
                       from_="+12056193926",
                       body="Violence Detected")